# NN Pytorch

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


## DATASET LOADING / SAVING

In [11]:
X = pd.read_csv('/home/singhaniatanay18/X_resampled.csv')
Y = pd.read_csv('./Y_resampled.csv',header=None)

In [12]:
print(X.shape)
print(Y.shape)

(7477003, 35)
(7477003, 1)


In [13]:
catenc = pd.factorize(Y.iloc[:,0])

In [14]:
Y2 = pd.DataFrame(data=catenc[0])
print(Y2.shape)

(7477003, 1)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y2, test_size=0.33, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5009592, 35)
(5009592, 1)
(2467411, 35)
(2467411, 1)


In [16]:
Xtr_np = np.array(X_train)
Ytr_np = np.array(y_train)
print(Ytr_np.shape)
print(Xtr_np.shape)
print(type(Ytr_np[0][0]))

(5009592, 1)
(5009592, 35)
<class 'numpy.int64'>


In [19]:
train = torch.utils.data.TensorDataset(torch.Tensor(Xtr_np), torch.Tensor(Ytr_np))
torch.save(train,'train.pt')

In [20]:
Xte_np = np.array(X_test)
Yte_np = np.array(y_test)
test = torch.utils.data.TensorDataset(torch.Tensor(Xte_np), torch.Tensor(Yte_np))
torch.save(test,'test.pt')

## LOAD DATASET

In [2]:
tr = torch.load('train.pt')
te = torch.load('test.pt')

In [3]:
train_loader = torch.utils.data.DataLoader(tr, batch_size = 1024, shuffle = True)

In [4]:
test_loader = torch.utils.data.DataLoader(te, batch_size = 1024, shuffle = True)

In [5]:
print(len(train_loader))
print(len(test_loader))

4893
2410


In [10]:
print(tr[0])

(tensor([8.0000e+01, 5.0110e+06, 4.0000e+00, 6.5696e+02, 6.5696e+02, 0.0000e+00,
        1.6424e+02, 3.2848e+02, 1.9761e+03, 5.2546e+02, 1.5965e+00, 7.1585e+05,
        1.8912e+06, 5.0048e+06, 2.1727e+01, 2.0653e+03, 1.5030e+03, 3.6911e+02,
        5.0109e+06, 1.6703e+06, 2.8881e+06, 5.0052e+06, 9.2000e+01, 9.2000e+01,
        7.9825e-01, 0.0000e+00, 2.9256e+02, 4.4680e+05, 1.0000e+00, 0.0000e+00,
        1.0000e+00, 8.1920e+03, 2.2098e+02, 2.0000e+01, 0.0000e+00]), tensor([14.]))


In [12]:
scaler = StandardScaler()
# Fit on training set only.

scaler.fit(tr.tensors[0])
# Apply transform to both the training set and the test set.
train_scaled = scaler.transform(tr.tensors[0])
test__scaled = scaler.transform(te.tensors[0])

In [14]:
print(train_scaled.shape)

(5009592, 35)


## PCA

In [16]:
# Make an instance of the Model
pca = PCA(.95)

In [17]:
pca.fit(train_scaled)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [18]:
pca.n_components_

16

In [20]:
train_PCA = pca.transform(train_scaled)
test_PCA = pca.transform(test__scaled)
print(type(train_PCA))

<class 'numpy.ndarray'>


In [22]:
train2 = torch.utils.data.TensorDataset(torch.Tensor(train_PCA), torch.Tensor(tr.tensors[1]))
torch.save(train2,'train_PCA.pt')
test2 = torch.utils.data.TensorDataset(torch.Tensor(test_PCA), torch.Tensor(te.tensors[1]))
torch.save(test2,'test_PCA.pt')

In [8]:
tr2 = torch.load('train_PCA.pt')
te2 = torch.load('test_PCA.pt')
train_loader = torch.utils.data.DataLoader(tr2, batch_size = 2048, shuffle = True)
test_loader = torch.utils.data.DataLoader(te2, batch_size = 1024, shuffle = True)

In [9]:
print(len(train_loader))
print(len(test_loader))

2447
2410


## NEURAL NET

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 32)
        self.fc6 = nn.Linear(32, 30)
        self.fc7 = nn.Linear(30, 28)
        self.fc8 = nn.Linear(28, 28)
        self.fc9 = nn.Linear(28, 28)
        self.fc10 = nn.Linear(28, 15)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc9(x))
        x = self.fc10(x)
        return F.log_softmax(x, dim=1)

In [5]:
print(torch.cuda.is_available())
device = torch.device('cuda:0')
print(device)
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device('cpu')
    print('running on GPU')

True
cuda:0
running on GPU


In [6]:
net = Net().to(device)
print(net)

Net(
  (fc1): Linear(in_features=16, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=32, bias=True)
  (fc6): Linear(in_features=32, out_features=30, bias=True)
  (fc7): Linear(in_features=30, out_features=28, bias=True)
  (fc8): Linear(in_features=28, out_features=28, bias=True)
  (fc9): Linear(in_features=28, out_features=28, bias=True)
  (fc10): Linear(in_features=28, out_features=15, bias=True)
)


In [7]:
#loss_function = nn.NLLLoss()
loss_function = nn.MultiLabelMarginLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

## TRAIN

In [ ]:
total = len(train_loader)
loss_list = []
for epoch in range(100):
    i =0# 3 full passes over the data
    outer = tqdm(total=0, desc='Epoch', position=0)
    for data in train_loader:
        xx,yy = data# X is the batch of features, y is the batch of targets.
        #print(xx.size(1))
        xx = xx.to(device)
        yy = yy.to(device)
        net.zero_grad()  # sets gradients to 0 before loss calc. You will do this likely every step.
        output = net(xx.view(xx.size(0), -1))  # pass in the reshaped batch (recall they are 28x28 atm)
        #print('output nikla')
        #print(output.size(1))
        #print(yy.size(1))
        yy = yy.squeeze(1)
        loss = F.nll_loss(output, yy.long())  # calc and grab the loss value
        loss.backward()  # apply this loss backwards thru the network's parameters
        optimizer.step()  # attempt to optimize weights to account for loss/gradients
        loss_list.append(float(loss))
        outer.set_description_str(f'Epoch : {epoch} ; Curr itr : {i}/{total} ; Loss : {loss}')
        i +=1
    print(f'Epoch : {epoch} ; Curr itr : {i}/{total} ; Loss : {loss} ')

Epoch: 0it [00:00, ?it/s]46/2447 ; Loss : 0.06899018585681915: 0it [01:36, ?it/s] 

Epoch : 0 ; Curr itr : 2447/2447 ; Loss : 0.06899018585681915 


Epoch: 0it [00:00, ?it/s]46/2447 ; Loss : 0.11847180873155594: 0it [01:36, ?it/s] 

Epoch : 2 ; Curr itr : 2447/2447 ; Loss : 0.11847180873155594 


Epoch: 0it [00:00, ?it/s]46/2447 ; Loss : 0.022922059521079063: 0it [01:35, ?it/s]

...

Epoch: 0it [00:00, ?it/s]446/2447 ; Loss : 0.08414938300848007: 0it [01:35, ?it/s] 

Epoch : 31 ; Curr itr : 2447/2447 ; Loss : 0.08414938300848007 


Epoch: 0it [00:00, ?it/s]446/2447 ; Loss : 0.04923718050122261: 0it [01:34, ?it/s] 

Epoch : 32 ; Curr itr : 2447/2447 ; Loss : 0.04923718050122261 


Epoch : 33 ; Curr itr : 338/2447 ; Loss : 0.03797025606036186: 0it [00:13, ?it/s] 

In [30]:
torch.save(net,'MultiLabelMarginLoss3_PCA')

/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [31]:
correct = 0
total = 0
itr = 0
with torch.no_grad():
    ooo = tqdm(total=len(test_loader), desc='Itr', position=0)
    for data in test_loader:
        xx, y = data
        xx = xx.to(device)
        y = y.to(device)
        output = net(xx.view(xx.size(0), -1))
        y = y.squeeze(1)
        y = y.long()
        #print(output)
        
        for idx, i in enumerate(output):
            #print(torch.argmax(i), y[idx])
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
        ooo.set_description_str(f'Itr :{itr}/{len(test_loader)}')
        itr+=1

print("Accuracy: ", round(correct/total, 3))

Itr :2409/2410:   0%|          | 0/2410 [04:12<?, ?it/s]

Accuracy:  0.984
